<a href="https://colab.research.google.com/github/cshamruk/hello-world/blob/master/FLOKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())


In [0]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1VNqMKjtKXiVA8cjOC9vQjaFKzBGyBfwD2EKV4Y1BSCs/edit#gid=164487166b')

In [0]:
Price = dict()
Capacity = dict()
for each in wb.worksheet('S_prices').get_all_values():
  Price[each[0]] = [float(p.replace(',','.')) if p else 0. for p in each[1:]]
  Capacity[each[0]] = [1 if p else 0 for p in each[1:]]

I2K = dict() # equivalence classes conversion matrix
for each in wb.worksheet('I_to_K').get_all_values():
  I2K[each[0]] = [float(p.replace(',','.')) if p else 0. for p in each[1:]]

# procurement requirements
Requirements = dict()
for each in wb.worksheet('K_req').get_all_values():
  Requirements[each[0]] = [float(p.replace(',','.')) for p in each[1:]]

S_min_amounts = [float(q) for q in wb.worksheet('S_min_amount').get_all_values()[0]]
 


The code below declares the variables, the constraints and the objective function to be fed into a GLOP linear solver.

Mathematical formulation of the problem:

Each of $I$ possible items belongs to one of the $K$ item classes (categories), and can be purchased from one of $S$ suppliers. We use the following variables:

 - `Deal` - $d_s$ - 0/1 decision of whether to deal with supplier $s$,
 - `Q` - $ Q_{i s}$ - quantity of item $i$ purchased from supplier $s$, 
 - `Class` - $X_k$ - aggregate quantity of items of class $k$ we have purchased; it is linked to `Q` by the formula $$ X_k = \sum_i w_{k,i} \sum_s Q_{i s},$$ where $w_{k,i}$ are stored in `I2K`;
 - `Deviation` - $Z_k$ - by how much we are missing the target for item class $k$; it is related to `Class` by the formula $$ Z_k = | X_k - \underline{X}_k |$$ 
 - `Bill` - $C_s$ - the total amount of money paid to supplier $s$, related to quantities $Q_{i s}$ by the formula $$ C_s = \sum_i P_{i s} Q_{i s}$$

 In addition to the equations above, we introduce constraints

 - `'Minimum amounts'` - supplier $s$ deals with us if we spend at least $\underline{C}_s$ (stored in `S_min_amounts`) $$ C_s \geq \underline{C}_s \cdot d_s,$$
 - `'Max_Q'` - we cannot buy from supplier $s$ more of product $i$ than the quantity $\overline{Q}_{i s}$ he can deliver (I set the upper limit to 100 for the suppliers that do sell the item) $$ Q_{i s} \leq \overline{Q}_{i s} \cdot d_s, $$
 - target constraints are hardcoded into the domain of variable `Class` $$ X_k \in [\underline{X}_k \cdot r_k , \overline{X}_k ]$$, where $r_k = 0$ if we can drop category $k$ from the basket and $1$ otherwise.

 The objective is dual, to minimize cost and stay close to the target. To accomodate for the duality of the objective, we introduce a vector of Lagrange multipliers $\lambda_k > 0$  (`item_class_importance`) and write the objective as $$ \min_{Q, C, Z, X, d} \sum_s C_s + \sum_k \lambda_k Z_k.$$

 This is an instance of mixed integer programming problem, which is easily solvable by a range of commercial and open-source solvers. 


---



In [0]:
from ortools.linear_solver import pywraplp

def LinearProgrammingExample(item_class_importance=None):
    """Linear programming sample."""
    # Instantiate a Glop solver, naming it LinearExample.
    solver = pywraplp.Solver('ProcurementFragmented',
                             pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    S = len(S_min_amounts)
    K = len(Requirements['Target order from Client (quantities)'])
    if not item_class_importance:
      item_class_importance = [100.]*K

    # Initialize variables
    Deal = dict() # supplier toggle
    Q = dict() # item quantities
    Class = dict()  # item equivalence classes
    Deviation = dict() # deviation from target
    Bill = dict() # payment to the store

    Constraints = dict()
    Constraints['Minimum amounts'] = dict()
    Constraints['Max_Q'] = dict()
    Constraints['Min_Q'] = dict()
    Constraints['Bill definition'] = dict()
    
    for s in range(S): # for each shop

      # set Deal variable
      Deal[s] = solver.IntVar(0,1,f'Deal with supplier {s}') 

      # set the Bill variable
      Bill[s] = solver.NumVar(0.,solver.infinity(),f'Payment to supplier {s}') 
      Constraints['Bill definition'][s] = solver.Constraint(0., 0.)
      Constraints['Bill definition'][s].SetCoefficient(Bill[s], -1.)
      
      # We spend the required amount if we decide to deal with the supplier
      Constraints['Minimum amounts'][s] = solver.Constraint(0, solver.infinity())
      Constraints['Minimum amounts'][s].SetCoefficient(Deal[s], -S_min_amounts[s])
      Constraints['Minimum amounts'][s].SetCoefficient(Bill[s], 1.)

      for item in Price.keys():
        
        # set Purchase Quantity variable
        Q[item,s] = solver.IntVar(0,solver.infinity(), item+f' from Supplier {s}') 

        # Bill_s = sum( price_i_s * Q_i_s )
        Constraints['Bill definition'][s].SetCoefficient(Q[item,s],Price[item][s])

        # Buy from suppliers that we deal with and that can deliver 
        # and no more than total amount required
        Constraints['Max_Q'][item,s] = solver.Constraint(0., solver.infinity()) 
        Constraints['Max_Q'][item,s].SetCoefficient(Q[item,s], -1) 
        Constraints['Max_Q'][item,s].SetCoefficient(Deal[s], 100.*Capacity[item][s])
        
        
    Constraints['DeviationU'], Constraints['DeviationL'] = dict(), dict()
    Constraints['Class definition'] = dict()

    for k in range(K):

      target = Requirements['Target order from Client (quantities)'][k]
      buffer = Requirements['Buffer to increase (if needed)'][k]
      r = Requirements['Can item be dropped from basket?'][k]
      lower = target * r
      upper = target + buffer

      # Purchased quantity must be above target, or 0 if class can be dropped 
      # Purchased quantity must be below target + buffer 
      Class[k] = solver.NumVar(lower, upper,f'Quantity of items of class {k}')

      Constraints['Class definition'][k] = solver.Constraint(0,0)
      Constraints['Class definition'][k].SetCoefficient(Class[k], -1.)
      
      for each in Price.keys():
        if I2K[each][k]:
          for s in range(S):
            Constraints['Class definition'][k].SetCoefficient(Q[each, s], 1.)

      # Deviation definition : Deviation_k := | Q_k - target_k |
      Deviation[k] = solver.NumVar(0,max(target-lower,buffer),f'Excess/Lack of items of class {k}')
      
      Constraints['DeviationU'][k] = solver.Constraint(target, solver.infinity())
      Constraints['DeviationU'][k].SetCoefficient(Class[k], 1)
      Constraints['DeviationU'][k].SetCoefficient(Deviation[k], -1)

      Constraints['DeviationL'][k] = solver.Constraint(target, solver.infinity())
      Constraints['DeviationL'][k].SetCoefficient(Class[k], 1.)
      Constraints['DeviationL'][k].SetCoefficient(Deviation[k], 1.)
      

    objective = solver.Objective()
    for s in range(S):
      objective.SetCoefficient(Bill[s], 1.)
    for k in range(K):
      objective.SetCoefficient(Deviation[k], item_class_importance[k])
    objective.SetMinimization()

    # Solve the system.
    solver.Solve()
    opt_solution = sum([Bill[s].solution_value() for s in range(S)])
    print('Number of variables =', solver.NumVariables())
    print('Number of constraints =', solver.NumConstraints())

    if any([Deviation[k].solution_value() for k in range(K)]):
      print('Targets omitted:\n')
      for k in range(K):
        if Deviation[k].solution_value():
          print(f'\tClass {k} : {Class[k].solution_value()}\n')
      
    # The value of each variable in the solution.
    print('=================\nSolution:')
    print('We do not deal with suppliers : ')
    print(','.join([f'{s}' for s in range(S) if (not Deal[s].solution_value())]))
    for s in range(S):
      if Bill[s].solution_value():
        print(f'\nSupplier {s} :')
        for key in Price.keys():
          if Q[key,s].solution_value():
            print('\t'+key+f' : {Q[key,s].solution_value():.0f}')
        print('====================')
        print(f'\tTotal cost : {Bill[s].solution_value():.2f} (Minimum : {S_min_amounts[s]:.2f})')
         
    # The objective value of the solution.
    print('====================')
    print('Overall cost =', opt_solution)
    return solver


In [79]:
slover = LinearProgrammingExample()

Number of variables = 509
Number of constraints = 531
Solution:
We do not deal with suppliers : 
0,7,9,10,11,12,13,14

Supplier 1 :
	APERITIVO CYNAR 900ML : 2
	APERITIVO MARTINI BITTER 995ML : 2
	CACHACA BOAZINHA 1L : 6
	GIN BULLDOG 750ML : 4
	GIN MONKEY 47 500ML : 5
	GIN PLYMOUTH 750ML : 7
	GIN SEAGERS NEGRONI 1L : 1
	GIN TANQUERAY 750ML : 1
	BITTER ANGOSTURA ORANGE 100ML : 3
	CACHACA BOAZINHA 600ML : 1
	Total cost : 2867.44 (Minimum : 250.00)

Supplier 2 :
	APERITIVO FERNET DUBAR 750ML : 6
	APERITIVO RAMAZZOTTI ROSATO 700ML : 4
	GIN GORDONS 750ML : 1
	CACHACA CHICO MINEIRO PRATA 600ML : 2
	Total cost : 372.29 (Minimum : 350.00)

Supplier 3 :
	GIN MARTIN MILLERS 700ML : 6
	Total cost : 997.80 (Minimum : 500.00)

Supplier 4 :
	APERITIVO FERNET BRANCA 750ML : 5
	GIN GORDONS 750ML : 2
	Total cost : 644.80 (Minimum : 600.00)

Supplier 5 :
	SKOL BEATS SENSES LATA 269ML : 8
	GIN GORDONS PINK 700ML : 5
	GIN NORDES 700ML : 3
	Total cost : 859.80 (Minimum : 700.00)

Supplier 6 :
	APERITIVO LIL

In [67]:
round(.3345320023098898,10)

0.3345320023